In [1]:
# Carga de paquetes necesarios para graficar
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Para leer archivos
import geopandas as gpd # Para hacer cosas geográficas
import seaborn as sns # Para hacer plots lindos
import networkx as nx # Construcción de la red en NetworkX
import scipy


# Preambulo

En esta sección cargamos los datos y los visualizamos. También construimos la matriz de adyacencia de la red de museos.

## Carga de datos de los museos

El listado de los museos, con el que se construye el [mapa](https://mapas.museosabiertos.org/museos/caba/), lo podemos encontrar [acá](https://github.com/MuseosAbiertos/Leaflet-museums-OpenStreetMap/blob/principal/data/export.geojson?short_path=bc357f3). También descargamos los barrios de CABA como complemento para los gráficos.

In [ ]:
# Leemos el archivo, retenemos aquellos museos que están en CABA, y descartamos aquellos que no tienen latitud y longitud
museos = gpd.read_file('https://raw.githubusercontent.com/MuseosAbiertos/Leaflet-museums-OpenStreetMap/refs/heads/principal/data/export.geojson')
barrios = gpd.read_file('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson')

## Visualización

In [ ]:
# Armamos el gráfico para visualizar los museos
fig, ax = plt.subplots(figsize=(10, 10))
barrios.boundary.plot(color='gray',ax=ax)
museos.plot(ax=ax)

## Cálculo de la matriz de distancias

Ahora construimos la matriz de distancias entre todos los museos. Como la tierra es un [geoide](https://es.wikipedia.org/wiki/Geoide) (es decir que no es [plana](https://es.wikipedia.org/wiki/Terraplanismo)), el cálculo de distancias no es una operación obvia. Una opción es proyectar a un [sistema de coordenadas local](https://geopandas.org/en/stable/docs/user_guide/projections.html), de forma tal que las distancias euclideas se correspondan con las distancias en metros. En este notebook usamos [EPSG](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset) 22184. 

In [ ]:
# En esta línea:
# Tomamos museos, lo convertimos al sistema de coordenadas de interés, extraemos su geometría (los puntos del mapa), 
# calculamos sus distancias a los otros puntos de df, redondeamos (obteniendo distancia en metros), y lo convertimos a un array 2D de numpy
D = museos.to_crs("EPSG:22184").geometry.apply(lambda g: museos.to_crs("EPSG:22184").distance(g)).round().to_numpy()

### Matriz de adyacencia: construimos una matriz conectando a cada museo con los $m$ más cercanos

In [ ]:
def construye_adyacencia(D,m): 
    # Función que construye la matriz de adyacencia del grafo de museos
    # D matriz de distancias, m cantidad de links por nodo
    # Retorna la matriz de adyacencia como un numpy.
    D = D.copy()
    l = [] # Lista para guardar las filas
    for fila in D: # recorriendo las filas, anexamos vectores lógicos
        l.append(fila<=fila[np.argsort(fila)[m]] ) # En realidad, elegimos todos los nodos que estén a una distancia menor o igual a la del m-esimo más cercano
    A = np.asarray(l).astype(int) # Convertimos a entero
    np.fill_diagonal(A,0) # Borramos diagonal para eliminar autolinks
    return(A)

m = 3 # Cantidad de links por nodo
A = construye_adyacencia(D,m)

## Construcción de la red en NetworkX (sólo para las visualizaciones)

In [ ]:
G = nx.from_numpy_array(A) # Construimos la red a partir de la matriz de adyacencia
# Construimos un layout a partir de las coordenadas geográficas
G_layout = {i:v for i,v in enumerate(zip(museos.to_crs("EPSG:22184").get_coordinates()['x'],museos.to_crs("EPSG:22184").get_coordinates()['y']))}

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) # Visualización de la red en el mapa
barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax) # Graficamos Los barrios
nx.draw_networkx(G,G_layout,ax=ax) # Graficamos los museos

# Resolución del TP

Aquí empieza la aventura... ¡diviertanse y consulten lo que necesiten!

## Punto 1:

Partimos de la ecuación 3:
$$
\textbf{p} = (1 - \alpha)\ C\ \textbf{p} + \frac{\alpha}{N}\ \textbf{1}
$$
Esto es lo mismo que
$$
\textbf{p} - (1 - \alpha)\ C\ \textbf{p} = \frac{\alpha}{N}\ \textbf{1}
$$
Sabiendo que $ I\ \textbf{p} = \textbf{p} $ sacamos de factor común el vector p
$$
(I - (1 - \alpha)\ C)\ \textbf{p} = \frac{\alpha}{N}\ \textbf{1}
$$
$$
\frac{N}{\alpha}(I - (1 - \alpha)\ C)\ \textbf{p} = \textbf{1}
$$
Como $ M = \frac{N}{\alpha}(I - (1 - \alpha)\ C) $ y $ \textbf{b} = \textbf{1} $ entonces la ecuacion resulta $ M\textbf{p} = \textbf{b} $
Como queriamos mostrar.

## Punto 2:
Recordemos que para que el sistema de ecuaciones lineales $ Ax = b $ tenga una única solución, $A$ debe ser inversible. Lo mismo ocurre con $ M\textbf{p} = \textbf{b}$. Para que sea inversible, $M$ debe ser cuadrada y sus filas deben ser linealmente independientes.
<br><br>
La matriz $C$ se calcula como $ C = A^\top\ K^{-1} $. Como $A \in \mathbb{R}^{N \times N}$ y $K$ es una matriz diagonal cuadrada, entonces $C$ también lo es. Como $M$ se construye restando un escalar multiplicando a $ C $ a una matriz identidad. A eso se le multiplica otro escalar. Por lo tanto $M$ es cuadrada.
<br><br>
Analizamos la singularidad de $M$. $ A $ es una matriz cuadrada donde su diagonal principal son ceros y sus elementos son unos o ceros. Lo que representa es el elemento $ a_{ij} $ es si el museo j se encuentra entre los 3 museos mas cercanos del i. Al calcular $C$ se transpone $A$ por lo tanto lo que representa eso mismo es el elemento $at_{ji}$
<br>
$K$ es una matriz diagonal con valores no nulos en la diagonal.
<br><br>
En este punto podemos afirmar que si $A$ es singular entonces $C$ es singular y por lo tanto $M$ es singular, debido a que el determinante de $C$ será $0$ y la operaciones que se realizan para construir $M$ haran que el determinante de $M$ sea igual a $0$.
<br><br>
Analizamos ahora la singularidad de $A$, Por la forma en la que se define, $A$ puede tener dos filas linealmente dependientes por lo que sería una matriz singular. Imaginemos el caso en que un museo (1) se encuentra en el cruce entre dos rectas perpendiculares entre si y sobre cada una de estas rectas se encuentran, en lados opuestos, dos museos. Estos 4 museos están a una distancia de una cuadra del museo A. El resto de los museos de la ciudad que no han sido descriptos se encuentran a, al menos, un km del museo (1). Podemos observar claramente como los museos que se sitúan sobre la misma recta y no son el (1), al representarlos en la matriz $A$ quedarían como filas linealmente dependientes. Por lo tanto $A$ es singular y no se cumplen las condiciones para garantizar que toda matriz $M$ tiene una única solución.

Cargamos funciones

In [ ]:
# Función para permutar filas (para descompPLU)
def permutacion(A, vector_P, index):
    n = A.shape[0]
    max_index = index + np.argmax(np.abs(A[index:, index]))
    #swap
    if max_index != index:
        A[[index, max_index]] = A[[max_index, index]]
        vector_P[[index, max_index]] = vector_P[[max_index, index]]


# Descomposición PLU con pivoteo
def calculaPLU(m, verbose=False):
    mc = m.copy().astype(np.float64)
    n = m.shape[0]
    P = np.eye(n)
    for i in range(n - 1):
        max_row = i + np.argmax(np.abs(mc[i:, i]))
        if max_row != i:
            mc[[i, max_row]] = mc[[max_row, i]]
            P[[i, max_row]] = P[[max_row, i]]
        a_ii = mc[i, i]
        if a_ii == 0:
            raise ValueError("Matriz singular (no invertible)")
        L_i = mc[i+1:, i] / a_ii
        mc[i+1:, i] = L_i
        mc[i+1:, i+1:] -= np.outer(L_i, mc[i, i+1:])
    
    L = np.tril(mc, -1) + np.eye(n)
    U = np.triu(mc)
    if verbose:
        print("P:\n", P)
        print("L:\n", L)
        print("U:\n", U)
    return P, L, U

def calcula_matrizK (A):
    n = A.shape[0]
    k = np.zeros((n,n),dtype=A.dtype)
    for i in range(n):
        k[i][i] = A[i].sum()
    
    return k


def norma2(v, l, r):
    if l+1 < r:
        mid = (l+r)//2
        n_left = norma2(v, l, mid)
        n_right = norma2(v, mid, r)
        return n_left + n_right
    else:
        return v[l]*v[l]

def norma2Vectorial(v):
    vc = v.copy()
    norma = norma2(vc,l=0,r=v.shape[0])    
    return np.sqrt(norma)



In [ ]:
# template de funciones 

def construye_adyacencia(D,m): 
    # Función que construye la matriz de adyacencia del grafo de museos
    # D matriz de distancias, m cantidad de links por nodo
    # Retorna la matriz de adyacencia como un numpy.
    D = D.copy()
    l = [] # Lista para guardar las filas
    for fila in D: # recorriendo las filas, anexamos vectores lógicos
        l.append(fila<=fila[np.argsort(fila)[m]] ) # En realidad, elegimos todos los nodos que estén a una distancia menor o igual a la del m-esimo más cercano
    A = np.asarray(l).astype(int) # Convertimos a entero
    np.fill_diagonal(A,0) # Borramos diagonal para eliminar autolinks
    return(A)

def calculaLU(matriz, verbose=False):
    mc = matriz.copy().astype(np.float64)
    n = matriz.shape[0]
    for i in range(n - 1):
        a_ii = mc[i, i]
        if a_ii == 0:
            raise ValueError("Cero en la diagonal durante LU (se requiere pivoteo)")
        L_i = mc[i+1:, i] / a_ii
        mc[i+1:, i] = L_i
        mc[i+1:, i+1:] -= np.outer(L_i, mc[i, i+1:])
    
    L = np.tril(mc, -1) + np.eye(n)
    U = np.triu(mc)
    if verbose:
        print("L:\n", L)
        print("U:\n", U)
    return L, U

# Función para calcular la inversa 
def inversa(m):
    n = m.shape[0]
    try:
        L, U = calculaLU(m)
        P = np.eye(n)  # Matriz de permutación identidad si no hay pivoteo
    except (ValueError, LinAlgError):
        P, L, U = calculaPLU(m)
    
    m_inv = np.zeros((n, n))
    for i in range(n):
        e_i = P.T @ np.eye(n)[:, i]  # Aplica la permutación P al vector canónico
        y = solve_triangular(L, e_i, lower=True)
        x = solve_triangular(U, y, lower=False)
        m_inv[:, i] = x
    return m_inv

def calcula_matriz_C(A): 
    # Función para calcular la matriz de trancisiones C
    # A: Matriz de adyacencia
    # Retorna la matriz C
    Kinv = inversa(calcula_matrizK(A))
    C = Kinv @ A
    return C


def calcula_pagerank(A,alfa):
    # Función para calcular PageRank usando LU
    # A: Matriz de adyacencia
    # d: coeficientes de damping
    # Retorna: Un vector p con los coeficientes de page rank de cada museo
    C = calcula_matriz_C(A)
    N = A.shape[0] # Obtenemos el número de museos N a partir de la estructura de la matriz A
    M = calcular_matriz_M(C, N, alfa)
    L, U = calculaLU(M) # Calculamos descomposición LU a partir de C y d
    b = ... # Vector de 1s, multiplicado por el coeficiente correspondiente usando d y N.
    Up = scipy.linalg.solve_triangular(L,b,lower=True) # Primera inversión usando L
    p = scipy.linalg.solve_triangular(U,Up) # Segunda inversión usando U
    return p

def calcula_matriz_C_continua(D): 
    # Función para calcular la matriz de trancisiones C
    # A: Matriz de adyacencia
    # Retorna la matriz C en versión continua
    D = D.copy()
    F = ...
    np.fill_diagonal(F,0)
    Kinv = inversa(calcula_matrizK(D)) # Calcula inversa de la matriz K, que tiene en su diagonal la suma por filas de F 
    C = Kinv @ F # Calcula C multiplicando Kinv y F
    return C




In [ ]:
def visualizar_p(A,size,alfa=1/5):
    C = calcula_matriz_C(A)
    N=A.shape[0]
    p = inversa(calcular_matriz_M(C, N, alfa)) @ np.ones(N)

    fig, ax = plt.subplots(figsize=(size['plot size'], size['plot size']))
    barrios.to_crs("EPSG:22184").boundary.plot(color='gray', linewidth=0.5, ax=ax)
    # Dibuja la red con tamaños proporcionales al PageRank
    node_sizes =  size['node size'] * p  # se puede ajustar tamaños
    nx.draw_networkx(
        G,
        pos=G_layout,
        ax=ax,
        node_size=node_sizes,  # escalado
        node_color='purple',      
        edge_color='blue',    
        width=0.5,            
        alpha=0.7,            
        with_labels=False     
    )
    
    ax.set_title("Red de Museos en CABA - Tamaño según PageRank (α={})".format(alfa), fontsize=16, pad=20)
    ax.grid(False)  # Ocultar cuadrícula
    
    plt.tight_layout()
    plt.show()

def visualizar_pR(D,G,size,m=3, alfa=1/5):
    A = construye_adyacencia(D,m)

    # Construcción de la red en NetworkX 
    G = nx.from_numpy_array(A) # Construimos la red a partir de la matriz de adyacencia
    # Construimos un layout a partir de las coordenadas geográficas
    G_layout = {i:v for i,v in enumerate(zip(museos.to_crs("EPSG:22184").get_coordinates()['x'],museos.to_crs("EPSG:22184").get_coordinates()['y']))}

    # Visualizacion
    fig, ax = plt.subplots(figsize=(15, 15)) # Visualización de la red en el mapa
    barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax) # Graficamos Los barrios
    nx.draw_networkx(G,G_layout,ax=ax) # Graficamos los museos

    C = calcula_matriz_C(A)
    N=A.shape[0]
    p = inversa(calcular_matriz_M(C, N, alfa)) @ np.ones(N)

    fig, ax = plt.subplots(figsize=(size['plot size'], size['plot size']))
    barrios.to_crs("EPSG:22184").boundary.plot(color='gray', linewidth=0.5, ax=ax)
    # Dibuja la red con tamaños proporcionales al PageRank
    node_sizes =  size['node size'] * p  # se puede ajustar tamaños
    nx.draw_networkx(
        G,
        pos=G_layout,
        ax=ax,
        node_size=node_sizes,  # escalado
        node_color='purple',      
        edge_color='blue',    
        width=0.5,            
        alpha=0.7,            
        with_labels=False     
    )
    
    ax.set_title("Red de Museos en CABA - Tamaño según PageRank (α={})".format(alfa), fontsize=16, pad=20)
    ax.grid(False)  # Ocultar cuadrícula
    
    plt.tight_layout()
    plt.show()


## Punto 3:

3.a) Visualizamos la red con los parametros:
* $m=3$
* $\alpha = \frac{1}{5}$


In [ ]:
size ={'node size':6000, 'plot size':10}
visualizar_p(A,size)

construimos una red con los siguientes paŕametros
* $m= 1,3,5,10$
* $\alpha = 1/5$

In [ ]:
for m in (1, 3, 5, 10):
    visualizar_pR(D, G, m=m,size=20)

* $m=5$
* $\alpha = 6/7, 4/5, 2/3, 1/2, 1/3, 1/5, 1/7$

In [ ]:
for alpha in [6/7, 4/5, 2/3, 1/2, 1/3, 1/5, 1/7]:
    visualizar_pR(D, G,size, m=5, alfa=alpha)


## Punto 4:

Recordemos: Siendo $C$ una matriz estocástica, y el vector $v_{0} \in \mathbb{R}^N$ que representa la distribución de museos que son la primer visita de los visitantes, y tiene en su elemento $i$ la cantidad de visitantes que tienen al museo $i$ como su primer opción. Entonces $v_{1} = Cv_{0}$.
<br><br>
Ejemplo: Quiero calcular $\textbf{w}$ luego que los visitantes recorran 3 museos. $r = 3$
<br>
$v_{1} = Cv_{0}$, $\ v_{2} = Cv_{1}$, $\ v_{3} = Cv_{2}$ 
<br><br>
$\textbf{w} = v_{0} + v_{1} + v_{2} = \sum_{i=0}^{2} v_{i}$
<br><br>
Si generalizamos a $r$ cantidad de visitas entonces $\textbf{w} = \sum_{i=0}^{r - 1} v_{i}$
<br><br>
Observación: $v_{2} = Cv_{1} = C\ C\ v_{0} = C^{2}v_{0}$. En general $v_{k} = C^{k}v_{0}$.
<br><br>
Por lo tanto, $\textbf{w} = \sum_{i=0}^{r - 1} C^{i}v_{0}$
<br><br>
Notemos que en este caso, por definición $v_{0} = \textbf{v}$. Luego $\textbf{w} = \sum_{i=0}^{r - 1} C^{i}\textbf{v}$
<br><br>
Llamemos $B = \sum_{i=0}^{r - 1} C^{i}$, la suma de matrices da como resultado una matriz y notar que $\textbf{v}$ no depende de la suma. Luego $\textbf{w} = B\textbf{v}$
<br><br>
Siempre que se cumpla que $\sum_{i=0}^{r - 1} C^{i}$ sea inversible, se pueden seguir esta serie de pasos:
<br><br>
$\textbf{w} = B\textbf{v}$
<br><br>
$B^{-1}\textbf{w} = B^{-1}B\textbf{v}$
<br><br>
$B^{-1}\textbf{w} = I\textbf{v}$
<br><br>
$B^{-1}\textbf{w} = \textbf{v}$
<br><br>
Concluimos lo que se quería mostrar.

## Punto 5:
### 5.a

In [ ]:
# La diafgonal son ceros
def calcula_matriz_C_continua_(D):
    N = D.shape[0]
    C = np.eye(N) # acá C es la matriz identidad
    M =D + C
    M = 1/M
    M -= C
    # De paso sumemos las columnas
    suma_columnas = np.transpose(np.add.reduce(M,axis=0))
    #y de paso multipliquuemos 
    for i in range(N):
        M[i] = M[i] * suma_columnas
    return np.transpose(M)

### 5.b

In [ ]:
def calcula_B(C,cantidad_de_visitas):
    # Recibe la matriz T de transiciones, y calcula la matriz B que representa la relación entre el total de visitas y el número inicial de visitantes
    # suponiendo que cada visitante realizó cantidad_de_visitas pasos
    # C: Matirz de transiciones
    # cantidad_de_visitas: Cantidad de pasos en la red dado por los visitantes. Indicado como r en el enunciado
    # Retorna:Una matriz B que vincula la cantidad de visitas w con la cantidad de primeras visitas v
    B = np.eye(C.shape[0])
    for i in range(cantidad_de_visitas-1):
        B = B + np.linalg.matrix_power(C, i) # Sumamos las matrices de transición para cada cantidad de pasos
    return B

### 5.c

In [ ]:
path
w = np.loadtxt(path+'visitas.txt', dtype=float)
C = calcula_matriz_C_continua_(D)
B = calcula_B(C, 3)
v = inversa(B) @ w
norma_v = norma2Vectorial(v)
print("La cantidad de visitantes que entraron en la red luego de dar 3 paso es: {}".format(norma_v))

## Punto 6:

# Extras

Para graficar la red con un conjunto de puntajes (como el Page Rank)

In [ ]:
factor_escala = 1e4 # Escalamos los nodos 10 mil veces para que sean bien visibles
fig, ax = plt.subplots(figsize=(10, 10)) # Visualización de la red en el mapa
barrios.to_crs("EPSG:22184").boundary.plot(color='gray',ax=ax) # Graficamos Los barrios
pr = np.random.uniform(0,1,museos.shape[0])# Este va a ser su score Page Rank. Ahora lo reemplazamos con un vector al azar
pr = pr/pr.sum() # Normalizamos para que sume 1
Nprincipales = 5 # Cantidad de principales
principales = np.argsort(pr)[-Nprincipales:] # Identificamos a los N principales
labels = {n: str(n) if i in principales else "" for i, n in enumerate(G.nodes)} # Nombres para esos nodos
nx.draw_networkx(G,G_layout,node_size = pr*factor_escala, ax=ax,with_labels=False) # Graficamos red
nx.draw_networkx_labels(G, G_layout, labels=labels, font_size=6, font_color="k") # Agregamos los nombres